<a href="https://colab.research.google.com/github/prem-cre/Multirag/blob/main/MultimodalAgents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Install Required Libraries
!pip install -qU langchain langchain_groq langchain_huggingface
!pip install -qU langchain-community langchain-google-community
!pip install -qU tavily-python wikipedia-api beautifulsoup4 requests
!pip install -qU tiktoken
!pip install -qU lxml[html_clean]
!pip install -qU faiss-cpu pypdf tiktoken tavily-python
!pip install -qU wikipedia # Added wikipedia package
!pip install langchain-google-genai


In [16]:

# Cell 2: Core Imports and Configuration
import os
import re
import json
import hashlib

from datetime import datetime
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from enum import Enum
from dataclasses import field

from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.documents import Document
from langchain_core.tools import tool
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.memory import ConversationBufferWindowMemory
from tavily import TavilyClient
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI


# Configure API Keys
os.environ["GROQ_API_KEY"] = userdata.get('groq_api_key')
os.environ["TAVILY_API_KEY"] = userdata.get('tavily')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_CSE_ID"] = userdata.get('GOOGLE_CSE_ID')

# Initialize services
# llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0.1)
# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# Initialize Gemini service (hypothetical example)
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # 👈 switch from "gemini-pro"
    temperature=0.1
)


tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
google_search_wrapper = GoogleSearchAPIWrapper(k=7)
wikipedia_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=1000)
wikipedia_tool = WikipediaQueryRun(api_wrapper=wikipedia_wrapper)

print("✅ All services initialized successfully!")

✅ All services initialized successfully!


In [8]:
# @title Enhanced Data Models for Legal Research

class SourceCredibility(Enum):
    """Credibility levels for sources"""
    OFFICIAL = "official"
    ACADEMIC = "academic"
    REPUTABLE = "reputable"
    GENERAL = "general"
    UNVERIFIED = "unverified"

@dataclass
class LegalSource:
    """Represents a legal source with metadata"""
    url: str
    title: str
    content: str
    credibility: SourceCredibility
    # date_accessed: datetime = field(default_factory=datetime.now)
    # date_published: Optional[str] = None
    author: Optional[str] = None
    jurisdiction: Optional[str] = None
    citation: Optional[str] = None
    hash: Optional[str] = None
    relevance_score: float = 0.0

    def __post_init__(self):
        self.hash = hashlib.md5(self.content.encode()).hexdigest()[:8]

@dataclass
class EvidenceItem:
    """Represents a piece of evidence in the legal research"""
    claim: str
    supporting_sources: List[LegalSource]
    confidence_score: float
    reasoning: str
    contradictions: List[Dict[str, Any]] = field(default_factory=list)
    verification_status: str = "pending"
    legal_basis: Optional[str] = None

@dataclass
class LegalResearchResult:
    """Complete legal research result with chain of evidence"""
    query: str
    summary: str
    evidence_chain: List[EvidenceItem]
    legal_precedents: List[Dict[str, Any]]
    jurisdictional_notes: Dict[str, str]
    confidence_assessment: Dict[str, float]
    citations: List[str]
    # timestamp: datetime = field(default_factory=datetime.now)

In [10]:
# Cell 3: Define Enhanced Research Tools

@tool
def legal_document_search(query: str, jurisdiction: str = "Indian") -> str:
    """
    Search legal documents, cases, and statutes with enhanced Indian law focus.
    Returns relevant legal information.
    """
    try:
        # Enhanced search query for Indian legal context
        legal_query = f"{jurisdiction} law legal {query} case judgment statute act"

        results = tavily_client.search(
            query=legal_query,
            search_depth="advanced",
            max_results=8,
            include_domains=["indiankanoon.org", "scconline.com", "lawmin.gov.in", "legislative.gov.in"],
        )

        # Return the raw results directly without formatting or filtering citations
        return json.dumps(results.get('results', []), indent=2)
    except Exception as e:
        return json.dumps({"error": f"Error searching legal documents: {str(e)}"})

@tool
def google_legal_search(query: str) -> str:
    """
    Calls Google Search API for specialized legal search focusing on Indian legal databases.
    Searches specifically in indiankanoon.org and scconline.com domains.
    Returns relevant legal information.
    """
    try:
        # Add site restrictions for Indian legal databases
        search_query = f"{query} site:indiankanoon.org OR site:scconline.com"
        search_results = google_search_wrapper.results(search_query, num_results=10)

        # Return the raw results directly
        return json.dumps(search_results, indent=2)
    except Exception as e:
        return f"Error calling Google Search: {str(e)}"

@tool
def wikipedia_legal_concepts(query: str) -> str:
    """
    Searches Wikipedia for legal concepts, landmark cases, and constitutional matters.
    Provides background information and case summaries.
    Returns the raw content.
    """
    try:
        wiki_result = wikipedia_tool.invoke(query)

        # Return the raw wiki result
        return wiki_result
    except Exception as e:
        return f"Error calling Wikipedia: {str(e)}"""



@tool
def fact_check_legal_claim(claim: str) -> str:
    """
    Comprehensive fact-checking of legal claims with Indian law focus.
    Returns detailed verification data.
    """
    try:
        # Enhanced queries for Indian legal context
        supporting_query = f'"{claim}" Indian law legal valid true correct Supreme Court High Court'
        contradicting_query = f'"{claim}" Indian law legal invalid false incorrect exception limitation'

        supporting = tavily_client.search(supporting_query, max_results=6)
        contradicting = tavily_client.search(contradicting_query, max_results=4)

        # Return the raw search results for supporting and contradicting evidence
        result = {
            'claim': claim,
            'supporting_evidence_raw': supporting.get('results', []),
            'contradicting_evidence_raw': contradicting.get('results', []),
            'verification_summary': f"Found {len(supporting.get('results', []))} potential supporting and {len(contradicting.get('results', []))} potential contradicting sources"
        }

        return json.dumps(result, indent=2)
    except Exception as e:
        return json.dumps({"error": f"Error fact-checking claim: {str(e)}"})



In [11]:
# @title Test Tool Output (Raw) for all Tools

# Define test queries relevant to each tool
test_queries = {
    "legal_document_search": "recent Supreme Court judgment on fundamental rights",
    "google_legal_search": "Maharashtra Rent Control Act 1999",
    "wikipedia_legal_concepts": "Doctrine of Basic Structure Indian Constitution",
    "verify_legal_citation": "AIR 2020 SC 123", # Example valid citation
    "fact_check_legal_claim": "Section 498A IPC is non-bailable",
    # Removed test for extract_legal_precedents
}

print("--- Testing Individual Tool Outputs ---")
print("Note: Examining raw output before LLM processing.")
print("-" * 40)

# Test legal_document_search tool
print("\nTesting legal_document_search...")
query = test_queries["legal_document_search"]
print(f"Query: {query}")
search_output = legal_document_search.invoke(query)
print("Raw Output:")
print(search_output[:100] + "..." if len(search_output) > 1000 else search_output) # Print truncated output for brevity
print("-" * 20)

# Test google_legal_search tool
print("\nTesting google_legal_search...")
query = test_queries["google_legal_search"]
print(f"Query: {query}")
google_output = google_legal_search.invoke(query)
print("Raw Output:")
print(google_output[:100] + "..." if len(google_output) > 1000 else google_output)
print("-" * 20)

# Test wikipedia_legal_concepts tool
print("\nTesting wikipedia_legal_concepts...")
query = test_queries["wikipedia_legal_concepts"]
print(f"Query: {query}")
wiki_output = wikipedia_legal_concepts.invoke(query)
print("Raw Output:")
print(wiki_output[:1000] + "..." if len(wiki_output) > 1000 else wiki_output)
print("-" * 20)



# Test fact_check_legal_claim tool
print("\nTesting fact_check_legal_claim...")
claim = test_queries["fact_check_legal_claim"]
print(f"Claim: {claim}")
fact_check_output = fact_check_legal_claim.invoke(claim)
print("Raw Output:")
print(fact_check_output[:100] + "..." if len(fact_check_output) > 1000 else fact_check_output)
print("-" * 20)


print("\n--- Individual Tool Testing Complete ---")

--- Testing Individual Tool Outputs ---
Note: Examining raw output before LLM processing.
----------------------------------------

Testing legal_document_search...
Query: recent Supreme Court judgment on fundamental rights
Raw Output:
[
  {
    "url": "https://indiankanoon.org/search/?formInput=infringement%20of%20fundamental%20right...
--------------------

Testing google_legal_search...
Query: Maharashtra Rent Control Act 1999
Raw Output:
[
  {
    "title": "Section 16 in The Maharashtra Rent Control Act, 1999",
    "link": "https://indi...
--------------------

Testing wikipedia_legal_concepts...
Query: Doctrine of Basic Structure Indian Constitution
Raw Output:
Page: Basic structure doctrine
Summary: The basic structure doctrine is a common law legal doctrine that the constitution of a sovereign state has certain characteristics that cannot be erased by its legislature. The doctrine is recognised in India, Bangladesh, Pakistan, and  Uganda. It was developed by the Supreme Court of

In [18]:
# @title Enhanced Legal Research Agent with Structured Output (Fixed)

class EnhancedLegalResearchAgent:
    """Advanced legal research agent with structured analysis and strong prompting"""

    def __init__(self, llm, tools):
        self.llm = llm
        self.tools = tools
        self.memory = ConversationBufferWindowMemory(
            memory_key="chat_history",
            return_messages=True,
            k=10
        )

        # Get tool names for the prompt
        tool_names = [tool.name for tool in tools]
        tool_descriptions = "\n".join([f"- {tool.name}: {tool.description}" for tool in tools])

        # Enhanced comprehensive prompt with structured output requirements
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an expert Indian legal research assistant with comprehensive knowledge of Indian law, statutes, and legal procedures. You provide thorough, well-structured legal analysis.

## YOUR APPROACH TO LEGAL RESEARCH:

1. **Query Analysis**: Identify legal issues, applicable laws, jurisdiction, and key legal concepts
2. **Comprehensive Research**: Search relevant cases, statutes, acts, and legal documents
3. **Source Verification**: Verify credibility and authenticity of all sources
4. **Precedent Analysis**: Extract and analyze relevant legal precedents and landmark cases
5. **Fact Verification**: Cross-check all legal claims against multiple authoritative sources
6. **Evidence Synthesis**: Build a logical chain of evidence with proper legal reasoning
7. **Confidence Assessment**: Evaluate the strength and reliability of findings

## AVAILABLE TOOLS:
{tool_descriptions}

You have access to the following tools: {tool_names}

## STRUCTURED OUTPUT FORMAT:

Your final analysis MUST be structured with these EXACT headings:




### 6. CITATIONS AND REFERENCES
- Minimum 5-6 authoritative citations
- Format: Case name, Citation, Court, Year
- Include statutory references
- Academic sources (if used)



Use the tools systematically to gather comprehensive information before providing your structured analysis."""),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])

        # Partial the prompt with tool information
        self.prompt = self.prompt.partial(
            tool_names=", ".join(tool_names),
            tool_descriptions=tool_descriptions,
            tools=tool_descriptions  # For backward compatibility
        )

        # Create the agent
        # Ensure handle_parsing_errors is set to True and return_intermediate_steps is True
        self.agent = create_react_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=self.prompt
        )

        self.executor = AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            memory=self.memory,
            verbose=True,
            max_iterations=20,
            handle_parsing_errors=True, # Keep this
            return_intermediate_steps=True # Keep this
        )

    def research(self, query: str) -> LegalResearchResult:
        """Conduct comprehensive legal research"""
        try:
            # Execute the research
            # Pass input as a dictionary
            result = self.executor.invoke({"input": query})

            # Parse and structure the results
            return result
        except Exception as e:
            print(f"Error in legal research: {str(e)}")
            return LegalResearchResult(
                query=query,
                summary=f"Error conducting research: {str(e)}",
                evidence_chain=[],
                legal_precedents=[],
                jurisdictional_notes={},
                confidence_assessment={"overall": 0.0},
                citations=[]
            )




In [13]:
# @title Enhanced Multi-Step Fact-Checking Process with LangChain Chains

from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

class EnhancedLegalFactChecker:
    """Enhanced fact-checking system using LangChain chains"""

    def __init__(self, llm, tools):
        self.llm = llm
        self.tools = tools

        # Create specialized chains for different steps
        self._create_analysis_chains()

    def _create_analysis_chains(self):
        """Create LangChain chains for structured analysis"""

        # Chain for initial claim analysis
        self.claim_analysis_chain = LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["claim"],
                template="""Analyze this legal claim in detail:

Claim: {claim}

Provide a structured analysis with:
1. **Main Legal Assertion**: What is the core legal claim?
2. **Jurisdiction**: Which legal system/jurisdiction applies?
3. **Legal Concepts**: What legal principles are involved?
4. **Factual Elements**: What specific facts are claimed?
5. **Potential Issues**: Any ambiguities or concerns?

Format your response as a detailed legal analysis."""
            )
        )

        # Chain for synthesizing evidence
        self.evidence_synthesis_chain = LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["evidence", "claim"],
                template="""Synthesize the following evidence for the legal claim:

Claim: {claim}

Evidence Found:
{evidence}

Provide:
1. **Strength of Evidence**: How strong is the supporting evidence?
2. **Contradictions**: Any conflicting information?
3. **Gaps**: What information is missing?
4. **Overall Assessment**: Your professional legal opinion

Be thorough and cite specific sources."""
            )
        )

        # Chain for final verification report
        self.final_report_chain = LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["claim", "analysis", "evidence", "confidence"],
                template="""Generate a comprehensive legal fact-checking report:

**CLAIM UNDER REVIEW**: {claim}

**INITIAL ANALYSIS**: {analysis}

**EVIDENCE SUMMARY**: {evidence}

**CONFIDENCE LEVEL**: {confidence}

Structure your report with these sections:

## 1. EXECUTIVE SUMMARY
- Brief overview of findings
- Verification status (Verified/Partially Verified/Unverified/False)

## 2. DETAILED LEGAL ANALYSIS
- Applicable laws and statutes
- Relevant case law
- Legal principles involved

## 3. EVIDENCE EVALUATION
- Supporting evidence strength
- Contradicting evidence analysis
- Source credibility assessment

## 4. LEGAL CITATIONS
- List all relevant citations found
- Include case names, citations, and years **IMPORTANT MANDATORY**

## 5. CONCLUSION AND CONFIDENCE ASSESSMENT
- Final determination
- Confidence percentage with reasoning
- Recommendations for further verification if needed

Ensure all citations follow proper legal citation format."""
            )
        )

    def verify_claim(self, claim: str) -> Dict[str, Any]:
        """Execute enhanced multi-step fact-checking process"""
        results = {
            "claim": claim,
            "timestamp": datetime.now(),
            "steps": []
        }

        try:
            # Step 1: Initial Claim Analysis using LangChain
            print("Step 1: Analyzing claim structure...")
            claim_analysis = self.claim_analysis_chain.run(claim=claim)
            results["steps"].append({
                "step": "claim_analysis",
                "output": claim_analysis,
                "timestamp": datetime.now()
            })

            # Step 2: Fact-check the claim using tool
            print("Step 2: Fact-checking claim...")
            fact_check_result = fact_check_legal_claim.invoke(claim)
            fact_check_data = json.loads(fact_check_result)
            print(fact_check_data)
            results["steps"].append({
                "step": "fact_checking",
                "output": fact_check_data,
                "timestamp": datetime.now()
            })

            # Step 3: Verify sources credibility
            print("Step 3: Verifying source credibility...")
            credible_sources = []
            all_sources = (
                fact_check_data.get('supporting_evidence', []) +
                fact_check_data.get('contradicting_evidence', [])
            )



            results["steps"].append({
                "step": "source_verification",
                "output": all_sources,
                "timestamp": datetime.now()
            })

            # Step 4: Search for legal precedents
            print("Step 4: Searching for legal precedents...")
            precedent_search = legal_document_search.invoke(claim, "Indian")
            precedent_data = json.loads(precedent_search)
            results["steps"].append({
                "step": "precedent_search",
                "output": precedent_data,
                "timestamp": datetime.now()
            })

            # Step 5: Synthesize evidence
            print("Step 5: Synthesizing evidence...")
            evidence_summary = json.dumps({
                "supporting": len(fact_check_data.get('supporting_evidence', [])),
                "contradicting": len(fact_check_data.get('contradicting_evidence', [])),
                "credible_sources": len([s for s in credible_sources if s.get('trusted', False)]),
                "precedents_found": len(precedent_data) if isinstance(precedent_data, list) else 0
            })

            synthesis = self.evidence_synthesis_chain.run(
                evidence=evidence_summary,
                claim=claim
            )
            results["steps"].append({
                "step": "evidence_synthesis",
                "output": synthesis,
                "timestamp": datetime.now()
            })



            # Step 7: Generate final report
            print("Step 6: Generating final report...")
            final_report = self.final_report_chain.run(
                claim=claim,
                analysis=claim_analysis,
                evidence=synthesis
            )

            results["final_report"] = final_report
            results["verification_complete"] = True

        except Exception as e:
            results["error"] = str(e)
            results["verification_complete"] = False


        return results



In [14]:
# @title Create Summary Chain for Final Output
from langchain.chains.summarize import load_summarize_chain
from langchain_core.prompts import PromptTemplate

def create_summary_chain(llm):
    """Create a summarization chain using LangChain's built-in functionality"""
    return load_summarize_chain(
        llm,
        chain_type="map_reduce",
        return_intermediate_steps=True,
        map_prompt=PromptTemplate(
            template="""Summarize the following legal information:
{text}

Focus on:
- Key legal points
- Important citations
- Relevant precedents""",
            input_variables=["text"]
        ),
        combine_prompt=PromptTemplate(
            template="""Combine these legal summaries into a comprehensive overview:
{text}

Provide:
1. Main legal findings
2. Critical citations
3. Overall conclusion""",
            input_variables=["text"]
        )
    )

In [21]:
all_tools = [
    legal_document_search,
    google_legal_search,
    fact_check_legal_claim,
    wikipedia_legal_concepts

]

# @title Instantiate and Run the Legal Fact-Checker and Research Agent

print("Initializing Legal Fact-Checker and Research Agent...")
# Re-using the 'llm' and 'all_tools' defined in previous sections
fact_checker = EnhancedLegalFactChecker(llm=llm, tools=all_tools)
legal_research_agent = EnhancedLegalResearchAgent(llm=llm, tools=all_tools) # Also removed from agent tools
print("Legal Fact-Checker and Research Agent initialized.")

# --- Define a sample legal claim to fact-check and research ---
# Example 1: A claim that is likely true (or easily verifiable)
claim_1 = "In the United States, a contract requires an offer, acceptance, and consideration to be legally binding."
# Example 2: A more nuanced or potentially disputable claim
claim_2 = "A non-compete clause in an employment contract is generally unenforceable in California if it restricts an employee's ability to practice a lawful profession, trade, or business."
# Example 3: A claim that might have contradictions or require deeper analysis
claim_3 = "All verbal agreements for real estate sales are unenforceable in all U.S. states."
# Example 4: Indian Law Claim (matches previous example)
claim_4 = "The Supreme Court of India in Kesavananda Bharati case established that the basic structure of the Constitution cannot be amended by Parliament."


# @title Execute Comprehensive Legal Research (using the agent)
def execute_legal_research(claim, agent, fact_checker):
    """Execute comprehensive legal research with structured output"""

    print("\n" + "="*100)
    print("🔍 COMPREHENSIVE LEGAL RESEARCH REPORT")
    print("="*100)
    print(f"\n📋 **CLAIM UNDER EXAMINATION:**\n{claim}")
    print("\n" + "-"*100)

    # Step 1: Fact-checking
    print("\n⚖️ **EXECUTING MULTI-STEP FACT-CHECKING PROCESS...**\n")
    # Use the fact_checker directly for the fact-checking steps
    fact_check_result = fact_checker.verify_claim(claim)

    # Display fact-checking steps
    if fact_check_result.get("verification_steps"):
        for i, step in enumerate(fact_check_result["verification_steps"], 1):
            step_name = step.get('step', 'Unknown Step').replace("_", " ").title()
            print(f"\n📌 Step {i}: {step_name}")
            print("-" * 50)

            if step.get("error"):
                 print(f"  ❌ Error: {step['error']}")
            elif step.get("analysis"):
                # Corrected f-string syntax
                analysis_text = step['analysis']
                print(f"  **Analysis:**\n{analysis_text[:500]}..." if len(analysis_text) > 500 else analysis_text)
            elif step.get("sources_found"):
                 data = step["sources_found"]
                 print(f"✓ Supporting Evidence: {len(data.get('supporting_evidence_raw', []))} sources")
                 print(f"✗ Contradicting Evidence: {len(data.get('contradicting_evidence_raw', []))} sources")
                 print(f"📊 Initial Confidence: {data.get('verification_summary', 'N/A')}") # Display summary from tool output
            elif step.get("credible_sources"):
                 credible = len([s for s in step["credible_sources"] if s.get('trusted', False) or s.get('credibility') in ['official', 'academic', 'reputable']])
                 print(f"🔐 Credible Sources Verified: {credible}/{len(step['credible_sources'])}")
            elif step.get("contradictions_found") is not None:
                 print(f"  **Contradictions Found:** {step['contradictions_found']}")
            elif step.get("confidence_score") is not None:
                 print(f"  **Confidence Score for Step:** {step['confidence_score']:.2%}")


    # Display final confidence score from fact checker
    print(f"\n\n🎯 **FINAL FACT-CHECKING CONFIDENCE SCORE: {fact_check_result.get('final_confidence', 0):.2%}**")

    # Step 2: Legal Research (using the agent)
    print("\n\n" + "-"*100)
    print("📚 **CONDUCTING IN-DEPTH LEGAL RESEARCH (using Agent)...**\n")

    # The agent will use its tools to perform the research based on the query
    research_result = agent.research(claim)

    # Display structured legal research output from the agent's final answer
    print("\n" + "="*100)
    print("📑 **STRUCTURED LEGAL ANALYSIS (from Agent)**")
    print("="*100)

    # Parse and display the structured output from the agent's summary
    output_text = research_result.summary

    # Extract sections using regex or string parsing (adjust patterns if needed based on agent output)
    sections = {
        "1. LEGAL ISSUE IDENTIFICATION": r"### 1\. LEGAL ISSUE IDENTIFICATION(.*?)(?=###|$)",
        "2. APPLICABLE LAWS AND STATUTES": r"### 2\. APPLICABLE LAWS AND STATUTES(.*?)(?=###|$)",
        "3. JUDICIAL PRECEDENTS AND CASE LAW": r"### 3\. JUDICIAL PRECEDENTS AND CASE LAW(.*?)(?=###|$)",
        "4. LEGAL ANALYSIS AND INTERPRETATION": r"### 4\. LEGAL ANALYSIS AND INTERPRETATION(.*?)(?=###|$)",
        "5. CONCLUSIONS AND RECOMMENDATIONS": r"### 5\. CONCLUSIONS AND RECOMMENDATIONS(.*?)(?=###|$)",
        "6. CITATIONS AND REFERENCES": r"### 6\. CITATIONS AND REFERENCES(.*?)(?=###|$)"
    }

    for section_title, pattern in sections.items():
        match = re.search(pattern, output_text, re.DOTALL | re.IGNORECASE)
        if match:
            print(f"\n### {section_title}")
            print("-" * 50)
            content = match.group(1).strip()
            print(content if content else "No specific information found for this section.")
        else:
            # If structured format not found, display the relevant part of the output
            print(f"\n### {section_title}")
            print("-" * 50)
            print(f"Could not find section '{section_title}' in agent output. Full output snippet:")
            # Find a relevant snippet around where the section *should* be
            snippet_start = output_text.find(section_title) - 100
            snippet_end = output_text.find(section_title) + 200
            print(output_text[max(0, snippet_start):min(len(output_text), snippet_end)] + "...")


    # Display citations and evidence sources collected by the agent
    print("\n\n" + "="*100)
    print("📖 **LEGAL CITATIONS AND EVIDENCE SOURCES (from Agent)**")
    print("="*100)

    if research_result.citations:
        print("\n**Agent-Extracted Citations:**")
        for i, citation in enumerate(research_result.citations[:10], 1):
            print(f"{i}. {citation}")
    else:
        print("\nNo specific citations extracted by the agent.")

    if research_result.evidence_chain:
        print("\n**Evidence Sources Used by Agent (Sample):**")
        for i, evidence in enumerate(research_result.evidence_chain[:5], 1):
            print(f"\n{i}. Claim: {evidence.claim}")
            if evidence.supporting_sources:
                source = evidence.supporting_sources[0]
                print(f"   Source URL: {source.url}")
                print(f"   Credibility: {source.credibility.value}")
                print(f"   Relevance Score: {source.relevance_score:.2f}")
    else:
        print("\nNo evidence sources recorded by the agent.")


    # Final assessment summary
    print("\n\n" + "="*100)
    print("📊 **FINAL ASSESSMENT SUMMARY**")
    print("="*100)

    # You can combine insights from both fact_check_result and research_result here
    print(f"\n**Fact-Checking Confidence Level:** {fact_check_result.get('final_confidence', 0):.2%}")
    print(f"**Agent's Confidence Assessment:** {research_result.confidence_assessment.get('overall', 0):.2%}")
    print(f"**Agent's Research Completeness:** {research_result.confidence_assessment.get('completeness', 0):.2%}")
    print(f"**Agent's Source Quality Assessment:** {research_result.confidence_assessment.get('source_quality', 0):.2%}")

    # You could add a final summary generated by an LLM chain here if needed
    # Example: final_summary = summary_chain.run(text=research_result.summary + json.dumps(fact_check_result))
    # print("\n**Overall Summary:**\n", final_summary)


    return {
        "claim": claim,
        "fact_check_result": fact_check_result,
        "research_result": research_result
    }

# @title Run Legal Research on Selected Claim

# Select a claim to analyze
selected_claim = claim_4  # Using the Indian Law Claim

print("🏛️ INDIAN LEGAL RESEARCH SYSTEM")
print("="*100)
print(f"\n🔍 Analyzing: {selected_claim}\n")

# Execute the research
results = execute_legal_research(selected_claim, legal_research_agent, fact_checker)

# @title Additional Analysis Functions using LangChain (Optional)

# These functions can be used independently after execute_legal_research
# They are not part of the automated execute_legal_research flow above
# unless explicitly called within it.

# Example: Generate Legal Opinion based on results (requires a summary chain or direct LLM call)
# def generate_legal_opinion(llm, claim, research_results):
#     """Generate a formal legal opinion using LangChain"""
#     # This would need a prompt and potentially structured input from research_results
#     pass

# Example: Create Citation Formatter (if needed for final presentation)
# def format_citations_properly(citations):
#     """Format legal citations according to Indian legal citation standards"""
#     pass


# @title Save Research Results (Optional)

def save_research_results(results, filename="legal_research_report.json"):
    """Save the research results to a JSON file"""

    # Prepare data for JSON serialization - be mindful of complex objects
    save_data = {
        "timestamp": datetime.now().isoformat(),
        "claim": results["claim"],
        "fact_checking_confidence": results["fact_check_result"].get("final_confidence", 0),
        "fact_checking_steps_count": len(results["fact_check_result"].get("verification_steps", [])),
        "agent_research_summary": results["research_result"].summary,
        "agent_extracted_citations": results["research_result"].citations[:10] if results["research_result"].citations else [],
        "agent_evidence_sources_count": len(results["research_result"].evidence_chain),
        "agent_jurisdictional_notes": results["research_result"].jurisdictional_notes,
        "agent_confidence_assessment": results["research_result"].confidence_assessment,
        # Potentially include summarized steps from fact_check_result if needed
    }

    try:
        with open(filename, 'w') as f:
            json.dump(save_data, f, indent=2)
        print(f"\n✅ Research results summary saved to {filename}")
    except Exception as e:
        print(f"\n❌ Error saving research results: {str(e)}")


# Save the results (optional)
save_research_results(results)


print("\n\n" + "="*100)
print("✅ LEGAL RESEARCH AND FACT-CHECKING PROCESS COMPLETE")
print("="*100)
print("\nThe system has executed the legal research and fact-checking process.")
print("Review the output above for the detailed report, analysis, and confidence scores.")

Initializing Legal Fact-Checker and Research Agent...
Legal Fact-Checker and Research Agent initialized.
🏛️ INDIAN LEGAL RESEARCH SYSTEM

🔍 Analyzing: The Supreme Court of India in Kesavananda Bharati case established that the basic structure of the Constitution cannot be amended by Parliament.


🔍 COMPREHENSIVE LEGAL RESEARCH REPORT

📋 **CLAIM UNDER EXAMINATION:**
The Supreme Court of India in Kesavananda Bharati case established that the basic structure of the Constitution cannot be amended by Parliament.

----------------------------------------------------------------------------------------------------

⚖️ **EXECUTING MULTI-STEP FACT-CHECKING PROCESS...**

Step 1: Analyzing claim structure...
Step 2: Fact-checking claim...
{'claim': 'The Supreme Court of India in Kesavananda Bharati case established that the basic structure of the Constitution cannot be amended by Parliament.', 'supporting_evidence_raw': [{'title': '提交表单显示Please verify the CAPTCHA before proceed怎么办？', 'url': 'http